In [ ]:
from scipy.spatial.distance import directed_hausdorff
import nibabel as nib
import numpy as np
from pathlib import Path
from sklearn.metrics import mean_squared_error

In [ ]:
def hausdorff_distance(output, segment):
    hd1 = directed_hausdorff(output, segment)[0]
    hd2 = directed_hausdorff(segment, output)[0]
    return np.max([hd1, hd2])


def hausdorff_distance3D(output, segment):
    distances = []
    for i in range(segment.shape[0]):
        segment_slice = segment[i]
        output_slice = output[i]
        distance = hausdorff_distance(output_slice, segment_slice)
        distances.append(distance)
    return distances

In [ ]:
segment = nib.load('images/Brats18_2013_17_1/original/seg.nii.gz')
segment = segment.get_fdata()

In [ ]:
baseline = nib.load('images/Brats18_2013_17_1/baseline.nii.gz')
baseline = baseline.get_fdata()

In [ ]:
baseline_distances = hausdorff_distance3D(segment, baseline)

In [ ]:
CUBE_SIZE = 25
OFFSET = 25

x_count = int(baseline.shape[0] / OFFSET)
y_count = int(baseline.shape[1] / OFFSET)
z_count = int(baseline.shape[2] / OFFSET)

for mod in ['t1', 't1ce', 't2', 'flair']:
    distances = np.zeros((x_count, y_count, z_count))
    path = Path(f'images/Brats18_2013_17_1/{mod}_results')
    for file in path.iterdir():
        name = file.name.split('.')[0]
        coords = [int(coord) for coord in name.split('-')]
        output = nib.load(str(file)).get_fdata()
        file_distances = hausdorff_distance3D(segment, output)
        mse = mean_squared_error(file_distances, baseline_distances)
        distances[coords[0]][coords[1]][coords[2]] = mse

    np.save(f'images/Brats18_2013_17_1/{mod}.npy', distances)